### Collection into pandas df

In [ ]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
host_ip = '192.168.1.8'
#iniciando cliente mongo
client = MongoClient(host_ip, 27017)
#selecionando db
db = client['news_2018']
#selecionando coleções
oantagonista = db.get_collection('oantagonista')
# collection into pandas df
oantagonista = pd.DataFrame(list(oantagonista.find()))

### Updating news in collection

In [ ]:
for index, row in carta.iterrows():
    db['veja'].update_one({'_id' : row['_id']}, 
                         {'$set' : {'section' :row['section'],
                                    'arg' :row['arg'],
                                    'sen' :row['sen'],
                                    'val' :row['val'],
                                    'mod' :row['mod'],
                                    'pre' :row['pre']
                                   }
                         })

### Update news veja

In [ ]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

# iniciando cliente mongo
client = MongoClient('localhost', 27017)
# selecionando db
veja_db = client['veja']
news = client['news']
# selecionando coleções
veja_articles = veja_db['veja']
veja_comments = veja_db['vejaComments']
# updating articles
for article in veja_articles.find():
    try:
        news['veja'].insert_one(article)
    except:
        print("article duplicate " + article['_id'], end='\r')
# updating comments articles
for comment in veja_comments.find():
    try:
        news['vejaComments'].insert_one(comment)
    except:
        print("comment duplicate " + comment['_id'], end='\r')

### Collection MetaData - compute ext size and number of comments

In [ ]:
from pymongo import MongoClient
import datetime
import time
import math

client = MongoClient('localhost', 27017)
news_2018 = client['news_2018']
news = client['news']

index = 0
total = news['veja'].count()

for article in news['veja'].find(no_cursor_timeout=True).batch_size(5):
    if not math.isnan(float(article['date'])):
        if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) == 2018:

            # compute title size
            title_size = len(str(article['title']).split())
            # compute text size
            text_size = len(str(article['text']).split())
            # compute number of comments
            number_comments = news['vejaComments'].find({"id_article" : article['url']}).count()

            # insert article in news_2018
            news_2018['veja'].insert_one(article)

             # insert comments in news_2018
            for comment in news['vejaComments'].find({"id_article" : article['url']}, no_cursor_timeout=True).batch_size(5):
                news_2018['vejaComments'].insert_one(comment)

            # insert metadata
            news_2018['vejaMetaData'].insert_one({
                'article_url' : article['url'],
                'text_size' : text_size,
                'number_comments' : number_comments
            })
    
    # status
    index += 1
    print('progress {0:.4f} %'.format(index / total * 100), end='\r')

### Clean News - filter news 2010/2018

In [ ]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)
db = client.news
collections = ['carta_capital', 'estadao', 'folha', 'g1', 'oantagonista', 'oglobo', 'veja']

for collection in collections:
    print('\n' + collection + '\n')

    for article in db[collection].find():
        try:
            #process Article
            if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) < 2010:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})

            elif float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) > 2018:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})
        except:
            print('delete article except')
            db[collection].delete_one({'_id':article['_id']})


### Clean Comments - filter news 2010/2018

In [ ]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)
db = client.news
collections = ['carta_capital', 'estadao', 'folha', 'g1', 'oantagonista', 'oglobo', 'veja']

for collection in collections:
    print('\n' + collection + '\n')

    for article in db[collection].find():
        try:
            #process Article
            if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) < 2010:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})

            elif float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) > 2018:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})
        except:
            print('delete article except')
            db[collection].delete_one({'_id':article['_id']})


### Selec News - selecting news 2018

In [ ]:
from pymongo import MongoClient
import datetime
import time
import re

client = MongoClient('localhost', 27017)

news = client.news
news_2018 = client.news_2018
collections = ['carta_capital', 'estadao', 'folha', 'g1', 'oantagonista', 'oglobo', 'veja']
PONTUACAO = u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ%.]'

for collection in collections:

	print('\n' + collection + '\n')

	for article in news[collection].find():
		if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) == 2018:
			try:
				article['section'] = re.sub(PONTUACAO, ' ', str(article['section'])).lower()
				article['text'] = re.sub(PONTUACAO, ' ', str(article['text'])).lower()
		
				news_2018[collection].insert_one(article)
			except:
				print("Error")
